<a href="https://colab.research.google.com/github/AUnzhakova/Orbit7/blob/master/keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras.models import Model

Using TensorFlow backend.


In [4]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)
root_path = 'gdrive/My Drive/snacks/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# Размер изображений
img_width, img_height = 150, 150
# Путь к каталогу с изображениями для обучения
train_data_dir = 'train'
# Путь к каталогу с изображениями для валидации
validation_data_dir = 'val'
# Количество изображений для обучения
nb_train_samples = 4838
# Количество изображений для валидации
nb_validation_samples = 955
# Количество эпох
epochs = 5
# Размер выборки
batch_size = 16

In [0]:
# Загружаем сеть VGG16 без части, которая отвечает за классификацию
base_model = applications.VGG16(weights='imagenet', include_top=False)

In [0]:
# Добавляем слои классификации
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
# Выходной слой с двадцатью нейронами для классов
predictions = Dense(20, activation='softmax')(x)

In [0]:
# Составляем сеть из двух частей
model = Model(inputs=base_model.input, outputs=predictions)

In [0]:
# "Замораживаем" сверточные уровни сети VGG16
# Обучаем только вновь добавленные слои
for layer in base_model.layers:
    layer.trainable = False

In [0]:
# Компилируем модель
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [32]:
!ls 

 bin	   lib					        opt	     srv
 boot	   lib32				        proc	     swift
 content   lib64				        root	     sys
 datalab   __MACOSX				        run	     tmp
 dev	   media				        sbin	     tools
 etc	   mnt					        snacks	     usr
 home	  'open?id=1BgkDtaplk6dw0b65C9RczKzYdkZuIe9R'   snacks.zip   var


In [45]:
# Создаем генератор данных для обучения
datagen = ImageDataGenerator(rescale=1. / 255)
train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode= 'categorical')

Found 4838 images belonging to 20 classes.


In [46]:
# Создаем генератор данных для валидации
validation_generator = datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode= 'categorical')

Found 955 images belonging to 20 classes.


In [26]:
!wget https://wolverine.raywenderlich.com/books/mlt/snacks.zip

--2019-04-21 17:15:34--  https://wolverine.raywenderlich.com/books/mlt/snacks.zip
Resolving wolverine.raywenderlich.com (wolverine.raywenderlich.com)... 13.249.87.8, 13.249.87.117, 13.249.87.64, ...
Connecting to wolverine.raywenderlich.com (wolverine.raywenderlich.com)|13.249.87.8|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 110589632 (105M) [application/zip]
Saving to: ‘snacks.zip’

snacks.zip          100%[===================>] 105.47M  11.0MB/s    in 12s     

2019-04-21 17:15:47 (8.64 MB/s) - ‘snacks.zip’ saved [110589632/110589632]



In [42]:
ls

annotations-test.csv   annotations-val.csv  README.txt  train/
annotations-train.csv  credits.csv          test/       val/


In [33]:
!pip install -U -q PyDrive

    100% |████████████████████████████████| 993kB 24.6MB/s 


In [35]:
import zipfile, os
print ("Uncompressing zip file")
zip_ref = zipfile.ZipFile('snacks.zip', 'r')
zip_ref.extractall('SNACK/')
zip_ref.close()

Uncompressing zip file


In [47]:
# Обучаем модель с помощью генератора
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps = nb_validation_samples)

Epoch 1/5
4838/4838 [==============================] - 269s 56ms/step - loss: 0.9635 - acc: 0.7051 - val_loss: 2.3654 - val_acc: 0.5069
Epoch 2/5
4838/4838 [==============================] - 268s 55ms/step - loss: 0.0944 - acc: 0.9772 - val_loss: 3.8307 - val_acc: 0.4813
Epoch 3/5
4838/4838 [==============================] - 267s 55ms/step - loss: 0.0203 - acc: 0.9945 - val_loss: 4.4767 - val_acc: 0.4921
Epoch 4/5
4838/4838 [==============================] - 269s 56ms/step - loss: 0.0139 - acc: 0.9960 - val_loss: 4.8051 - val_acc: 0.4982
Epoch 5/5
4838/4838 [==============================] - 268s 55ms/step - loss: 0.0126 - acc: 0.9965 - val_loss: 5.1831 - val_acc: 0.4830
